In [ ]:
from IPython.display import display
import pandas as pd
import csv
import random
import re
# import pyperclip
import numpy as np
import seaborn as sns
import requests
import matplotlib, matplotlib.pyplot as plt
import pprint
import os
import dateutil
import datetime

data=pd.read_excel("赛题1测试数据集\年报-的对外提供保证担保信息.xlsx").drop_duplicates(keep='first')
                                                                 
data

In [ ]:
def get_number(cell):
    if not isinstance(cell, str):
        return cell
    _ = re.findall(r'[\d.]+',cell)
    if not _:
        return None
    return float(_[0])

In [ ]:
df_num=data[["企业编号",'年报年份','主债权数额']].replace("企业选择不公示",None).replace("万元",None)
df_num=df_num.applymap(get_number).groupby(["企业编号","年报年份"]).sum().unstack().T.reindex(columns=list(range(4001,4526)))
df_num

In [ ]:
for i in range(len(data)):
    if (type(data.iloc[i,0])==np.str and  data.iloc[i,0]!="企业选择不公示"):
        if(data.iloc[i,0][-2:]=="万元"):
            data.iloc[i,0]=float(data.iloc[i,0][:-2])*10000
        else:data.iloc[i,0]=float(data.iloc[i,0])
data

In [ ]:
def limit(cell):
    if cell > 1:
        return 1
    return cell

In [ ]:
df1=data.set_index("保证的期间").loc[["期限","企业选择不公示","限期","未约定"],:].reset_index().set_index("企业编号")


dummy_1=pd.get_dummies(df1["保证的期间"],prefix='保证的期间')
dummy_1=dummy_1.groupby(["企业编号"]).mean()
dummy_1["保证的期间_期限"]=dummy_1["保证的期间_期限"]+dummy_1["保证的期间_限期"]
dummy_1=dummy_1.drop(columns="保证的期间_限期").T
dummy_1=dummy_1.applymap(limit).reindex(columns=list(range(4001,4526))).fillna(0)
dummy_1

In [ ]:
df2=data.set_index("保证的方式").loc[["连带保证","企业选择不公示","一般保证","未约定"],:].reset_index().set_index("企业编号")


dummy_2=pd.get_dummies(df2["保证的方式"],prefix='保证的方式')
dummy_2=dummy_2.groupby(["企业编号"]).mean().T
dummy_2=dummy_2.applymap(limit).reindex(columns=list(range(4001,4526))).fillna(0)
dummy_2

In [ ]:
df3=data.set_index("主债权种类").loc[["合同","企业选择不公示","其他"],:].reset_index().set_index("企业编号")

dummy_3=pd.get_dummies(df3["主债权种类"],prefix='主债权种类')
dummy_3=dummy_3.groupby(["企业编号"]).mean().T
dummy_3=dummy_3.applymap(limit).reindex(columns=list(range(4001,4526))).fillna(0)
dummy_3

In [ ]:
result_12=dummy_1.append(dummy_2).append(dummy_3).fillna(0).append(df_num)#三表连接
result_12

In [ ]:
result_12.to_pickle('年报-的对外提供保证担保信息-testing.pickle')